# Problema

Predecir el tipo de flor

# El set de datos

Número de Instancias:

El conjunto de datos Iris consta de 150 muestras de flores Iris con 4 características cada una.

Características:

* Longitud del sépalo (sepal length)
* Ancho del sépalo (sepal width)
* Longitud del pétalo (petal length)
* Ancho del pétalo (petal width)

Clases o Etiquetas:

* Setosa
* Versicolor
* Virginica



In [1]:
# imports
from sklearn.datasets import load_iris
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('dataset_banco_clean.csv')
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143.0,yes,no,unknown,5,may,261.0,1,-1.0,0,unknown,no
1,44,technician,single,secondary,no,29.0,yes,no,unknown,5,may,151.0,1,-1.0,0,unknown,no
2,33,entrepreneur,married,secondary,no,2.0,yes,yes,unknown,5,may,76.0,1,-1.0,0,unknown,no
3,47,blue-collar,married,unknown,no,1506.0,yes,no,unknown,5,may,92.0,1,-1.0,0,unknown,no
4,33,unknown,single,unknown,no,1.0,no,no,unknown,5,may,198.0,1,-1.0,0,unknown,no


# Objetivo

Generar un model de clasificación capaz de predecir la clase de flor en función de las carácterísticas del dataset

* Aplicar las técnicas oportunas de procesamiento de datos

* Valorar diferentes modelos de clasificación

* Comparación entre modelos

* Ensemble

* Métricas

* Conclusiones finales

In [ ]:
missing_values = iris_df.isnull().sum()

# Muestra los valores faltantes
print("Valores faltantes por característica:")
print(missing_values)

# Si necesitas ver el número total de valores faltantes en todo el conjunto de datos, puedes sumar los resultados
total_missing = missing_values.sum()
print("\nNúmero total de valores faltantes:", total_missing)

Valores faltantes por característica:
sepal length (cm)    0
sepal width (cm)     0
petal length (cm)    0
petal width (cm)     0
target               0
dtype: int64

Número total de valores faltantes: 0


In [4]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn import metrics
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, mean_squared_error, mean_squared_log_error, roc_curve, auc
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as imbpipeline
from sklearn.svm import SVC

In [9]:
from sklearn.tree import DecisionTreeClassifier
from imblearn.pipeline import Pipeline as imbpipeline
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

X = df.drop('y', axis=1)
y = df['y']

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

numeric_features = ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']
categorical_features = ['marital', 'housing', 'loan', 'contact', 'job', 'month', 'poutcome', 'education']

numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

model = imbpipeline(steps=[
    ('preprocessor', preprocessor),
    ('sampler', SMOTE(random_state=42)),
    ('classifier', DecisionTreeClassifier(random_state=42))
])

# Hyperparameter tuning
param_grid = {
    'classifier__max_depth': [20, 30],
    'classifier__min_samples_split': [2],
    'classifier__min_samples_leaf': [1]
}

grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(x_train, y_train)

best_model = grid_search.best_estimator_

y_pred = best_model.predict(x_test)

# Evaluation
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

conf_matrix = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:\n', conf_matrix)

class_report = classification_report(y_test, y_pred)
print('Classification Report:\n', class_report)

# Display best hyperparameters
print('Best Hyperparameters:', grid_search.best_params_)

Accuracy: 0.87
Confusion Matrix:
 [[10941  1030]
 [  697   889]]
Classification Report:
               precision    recall  f1-score   support

          no       0.94      0.91      0.93     11971
         yes       0.46      0.56      0.51      1586

    accuracy                           0.87     13557
   macro avg       0.70      0.74      0.72     13557
weighted avg       0.88      0.87      0.88     13557

Best Hyperparameters: {'classifier__max_depth': 20, 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 2}


In [6]:
X = df.drop('y', axis=1)
y = df['y']

x_train, x_test_val, y_train, y_test_val = train_test_split(X, y, test_size=0.3, random_state=42)

numeric_features = ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']
categorical_features = ['marital', 'housing', 'loan', 'contact', 'job', 'month', 'poutcome', 'education']

numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

model = imbpipeline(steps=[
    ('preprocessor', preprocessor),
    ('sampler', SMOTE(random_state=42)),
    ('classifier', RandomForestClassifier(random_state=42))
])

# Hyperparameter tuning
param_grid = {
    'classifier__n_estimators': [100],
    'classifier__max_depth': [None],
    'classifier__min_samples_split': [2],
    'classifier__min_samples_leaf': [1]
}

grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(x_train, y_train)

best_model = grid_search.best_estimator_

y_pred = best_model.predict(x_test)

# Evaluation
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

conf_matrix = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:\n', conf_matrix)

class_report = classification_report(y_test, y_pred)
print('Classification Report:\n', class_report)

# Display best hyperparameters
print('Best Hyperparameters:', grid_search.best_params_)

Accuracy: 0.90
Confusion Matrix:
 [[5686  304]
 [ 372  416]]
Classification Report:
               precision    recall  f1-score   support

          no       0.94      0.95      0.94      5990
         yes       0.58      0.53      0.55       788

    accuracy                           0.90      6778
   macro avg       0.76      0.74      0.75      6778
weighted avg       0.90      0.90      0.90      6778

Best Hyperparameters: {'classifier__max_depth': None, 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 2, 'classifier__n_estimators': 100}


In [10]:
X = df.drop('y', axis=1)
y = df['y']

x_train, x_test_val, y_train, y_test_val = train_test_split(X, y, test_size=0.3, random_state=42)
x_test, x_val, y_test, y_val = train_test_split(x_test_val, y_test_val, test_size=0.5, random_state=42)

numeric_features = ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']
categorical_features = ['marital', 'housing', 'loan', 'contact', 'job', 'month', 'poutcome', 'education']

numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Create the KNN classifier
knn_model = imbpipeline(steps=[
    ('preprocessor', preprocessor),
    ('sampler', SMOTE(random_state=42)),  # SMOTE
    ('classifier', KNeighborsClassifier())
])

# Train the model on the training data
knn_model.fit(x_train, y_train)

# Make predictions on the validation set
y_val_pred = knn_model.predict(x_val)

# Evaluate the model on the validation set
accuracy = accuracy_score(y_val, y_val_pred)
conf_matrix = confusion_matrix(y_val, y_val_pred)
class_report = classification_report(y_val, y_val_pred)

# Display evaluation metrics
print("Validation Set:")
print(f'Accuracy: {accuracy:.2f}')
print('Confusion Matrix:\n', conf_matrix)
print('Classification Report:\n', class_report)

Validation Set:
Accuracy: 0.84
Confusion Matrix:
 [[5120  861]
 [ 200  598]]
Classification Report:
               precision    recall  f1-score   support

          no       0.96      0.86      0.91      5981
         yes       0.41      0.75      0.53       798

    accuracy                           0.84      6779
   macro avg       0.69      0.80      0.72      6779
weighted avg       0.90      0.84      0.86      6779



In [11]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(df['y'])

X = df.drop('y', axis=1)
y = y_encoded

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

numeric_features = ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']
categorical_features = ['marital', 'housing', 'loan', 'contact', 'job', 'month', 'poutcome', 'education']

numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

model = imbpipeline(steps=[
    ('preprocessor', preprocessor),
    ('sampler', SMOTE(random_state=42)),
    ('classifier', XGBClassifier(random_state=42))
])

# Hyperparameter tuning
param_grid = {
    'classifier__max_depth': [5],  # Example depths, adjust as needed
    'classifier__learning_rate': [0.1],
    'classifier__n_estimators': [100]
}

grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(x_train, y_train)

best_model = grid_search.best_estimator_

y_pred = best_model.predict(x_test)

# Evaluation
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

conf_matrix = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:\n', conf_matrix)

class_report = classification_report(y_test, y_pred)
print('Classification Report:\n', class_report)

# Display best hyperparameters
print('Best Hyperparameters:', grid_search.best_params_)


Accuracy: 0.89
Confusion Matrix:
 [[11039   932]
 [  504  1082]]
Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.92      0.94     11971
           1       0.54      0.68      0.60      1586

    accuracy                           0.89     13557
   macro avg       0.75      0.80      0.77     13557
weighted avg       0.91      0.89      0.90     13557

Best Hyperparameters: {'classifier__learning_rate': 0.1, 'classifier__max_depth': 5, 'classifier__n_estimators': 100}


## Ensemble de votación

In [12]:
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from imblearn.pipeline import Pipeline as imbpipeline
from imblearn.over_sampling import SMOTE

# Encode the target variable
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(df['y'])

# XGBoost Model
X_xgb = df.drop('y', axis=1)
y_xgb = y_encoded

x_train_xgb, x_test_xgb, y_train_xgb, y_test_xgb = train_test_split(X_xgb, y_xgb, test_size=0.3, random_state=42)

numeric_features_xgb = ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']
categorical_features_xgb = ['marital', 'housing', 'loan', 'contact', 'job', 'month', 'poutcome', 'education']

numeric_transformer_xgb = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer_xgb = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor_xgb = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer_xgb, numeric_features_xgb),
        ('cat', categorical_transformer_xgb, categorical_features_xgb)
    ])

model_xgb = imbpipeline(steps=[
    ('preprocessor', preprocessor_xgb),
    ('sampler', SMOTE(random_state=42)),
    ('classifier', XGBClassifier(random_state=42))
])

# Hyperparameter tuning for XGBoost
param_grid_xgb = {
    'classifier__max_depth': [5],  # Example depths, adjust as needed
    'classifier__learning_rate': [0.1],
    'classifier__n_estimators': [100]
}

grid_search_xgb = GridSearchCV(model_xgb, param_grid_xgb, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_xgb.fit(x_train_xgb, y_train_xgb)

best_model_xgb = grid_search_xgb.best_estimator_

y_pred_xgb = best_model_xgb.predict(x_test_xgb)

# Evaluation for XGBoost
accuracy_xgb = accuracy_score(y_test_xgb, y_pred_xgb)
print(f'XGBoost Accuracy: {accuracy_xgb:.2f}')

conf_matrix_xgb = confusion_matrix(y_test_xgb, y_pred_xgb)
print('XGBoost Confusion Matrix:\n', conf_matrix_xgb)

class_report_xgb = classification_report(y_test_xgb, y_pred_xgb)
print('XGBoost Classification Report:\n', class_report_xgb)

# Display best hyperparameters for XGBoost
print('XGBoost Best Hyperparameters:', grid_search_xgb.best_params_)

# Random Forest Model
X_rf = df.drop('y', axis=1)
y_rf = df['y']

x_train_rf, x_test_rf, y_train_rf, y_test_rf = train_test_split(X_rf, y_rf, test_size=0.3, random_state=42)

numeric_features_rf = ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']
categorical_features_rf = ['marital', 'housing', 'loan', 'contact', 'job', 'month', 'poutcome', 'education']

numeric_transformer_rf = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer_rf = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor_rf = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer_rf, numeric_features_rf),
        ('cat', categorical_transformer_rf, categorical_features_rf)
    ])

model_rf = imbpipeline(steps=[
    ('preprocessor', preprocessor_rf),
    ('sampler', SMOTE(random_state=42)),
    ('classifier', RandomForestClassifier(random_state=42))
])

# Hyperparameter tuning for Random Forest
param_grid_rf = {
    'classifier__n_estimators': [100],
    'classifier__max_depth': [None],
    'classifier__min_samples_split': [2],
    'classifier__min_samples_leaf': [1]
}

grid_search_rf = GridSearchCV(model_rf, param_grid_rf, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_rf.fit(x_train_rf, y_train_rf)

best_model_rf = grid_search_rf.best_estimator_

y_pred_rf = best_model_rf.predict(x_test_rf)

# Evaluation for Random Forest
accuracy_rf = accuracy_score(y_test_rf, y_pred_rf)
print(f'Random Forest Accuracy: {accuracy_rf:.2f}')

conf_matrix_rf = confusion_matrix(y_test_rf, y_pred_rf)
print('Random Forest Confusion Matrix:\n', conf_matrix_rf)

class_report_rf = classification_report(y_test_rf, y_pred_rf)
print('Random Forest Classification Report:\n', class_report_rf)

# Display best hyperparameters for Random Forest
print('Random Forest Best Hyperparameters:', grid_search_rf.best_params_)

# Ensemble by Voting
from sklearn.ensemble import VotingClassifier

voting_clf = VotingClassifier(estimators=[
    ('xgb', best_model_xgb),
    ('rf', best_model_rf)
], voting='soft')  # Soft voting for probability predictions

voting_clf.fit(x_train_xgb, y_train_xgb)
y_pred_voting = voting_clf.predict(x_test_xgb)

# Evaluation for Voting
accuracy_voting = accuracy_score(y_test_xgb, y_pred_voting)
print(f'Voting Ensemble Accuracy: {accuracy_voting:.2f}')

conf_matrix_voting = confusion_matrix(y_test_xgb, y_pred_voting)
print('Voting Ensemble Confusion Matrix:\n', conf_matrix_voting)

class_report_voting = classification_report(y_test_xgb, y_pred_voting)
print('Voting Ensemble Classification Report:\n', class_report_voting)


XGBoost Accuracy: 0.89
XGBoost Confusion Matrix:
 [[11039   932]
 [  504  1082]]
XGBoost Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.92      0.94     11971
           1       0.54      0.68      0.60      1586

    accuracy                           0.89     13557
   macro avg       0.75      0.80      0.77     13557
weighted avg       0.91      0.89      0.90     13557

XGBoost Best Hyperparameters: {'classifier__learning_rate': 0.1, 'classifier__max_depth': 5, 'classifier__n_estimators': 100}
Random Forest Accuracy: 0.90
Random Forest Confusion Matrix:
 [[11362   609]
 [  717   869]]
Random Forest Classification Report:
               precision    recall  f1-score   support

          no       0.94      0.95      0.94     11971
         yes       0.59      0.55      0.57      1586

    accuracy                           0.90     13557
   macro avg       0.76      0.75      0.76     13557
weighted avg       0.90      0.

In [14]:
# Ensemble by Voting
from sklearn.ensemble import VotingClassifier

voting_clf = VotingClassifier(estimators=[
    ('xgb', best_model_xgb),
    ('rf', best_model_rf)
], voting='soft')  # Soft voting for probability predictions

voting_clf.fit(x_train_xgb, y_train_xgb)
y_pred_voting = voting_clf.predict(x_test_xgb)

# Evaluation for Voting
accuracy_voting = accuracy_score(y_test_xgb, y_pred_voting)
print(f'Voting Ensemble Accuracy: {accuracy_voting:.2f}')

conf_matrix_voting = confusion_matrix(y_test_xgb, y_pred_voting)
print('Voting Ensemble Confusion Matrix:\n', conf_matrix_voting)

class_report_voting = classification_report(y_test_xgb, y_pred_voting)
print('Voting Ensemble Classification Report:\n', class_report_voting)


Voting Ensemble Accuracy: 0.90
Voting Ensemble Confusion Matrix:
 [[11220   751]
 [  578  1008]]
Voting Ensemble Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.94      0.94     11971
           1       0.57      0.64      0.60      1586

    accuracy                           0.90     13557
   macro avg       0.76      0.79      0.77     13557
weighted avg       0.91      0.90      0.90     13557



# Conclusiones

Vemos que todos los modelos son bastante precisos, sin embargo, el random forest resulta ser ligeramente mejor.

El ensemble de RF y XGB ofrece en este caso resultados parecidos al RF anterior